<span type="title">Java DataBase Connectivity</span> | <span type="update">2018-10-04</span> - Version <span type="version">1.0</span>
    
    
<span type="intro"><p class="card-text">本章主要介绍使用 JDBC API 操纵数据库的流程、方法和演变过程。JDBC 的包位于 javax.sql 中，其包含了 Connection、DataSource、Statement、ResultRow等常见的数据库操纵类。在前半部分，除了介绍基本的 JDBC，还介绍了大文件处理、SQL执行安全、线程池、事务、批处理的相关知识。同时介绍了 DatabaseMetaData, ResultSetMetaData 的使用。</p><p class="card-text">在后半部分介绍了不同工具实现的 Data Access Object 通用接口，同时介绍了在其中使用 BeanUtils 和 DBUtils 来健壮程序的方法，其中 BeanUtils 用来将数据库数据映射为 Java Bean（POJO），DBUtils 用来线程安全的进行增删改查。</p><p class="card-text">在实际开发中常常需要自己实现和编写 DAO，通过结合自编的 JDBCUtils 管理数据源、用户凭证、错误处理，使用 DBUtils（或许还包括BeanUtils），可以快速编写程序需要的 DAO 层。</p></span>

# JDBC 基础概念



## 获取数据库连接 Connection 对象

**直接使用驱动获取连接**

JDBC 是 Java 程序连接数据库的一组通用 API 规范。通过 JDBC 访问数据库不需要代码耦合任何专有数据库厂商的驱动程序，比如 Mysql 或者 Oracle，以及 Microsoft SQL Server。通过指定驱动程序 Driver、JDBC访问地址（包含数据库协议、子协议以及地址、端口和数据库）、用户名和密码即可通过 `Driver.conncet` 获得 Connection 对象。通过此对象操纵数据库的表，进行增删改查的操作。

下面展示了一段测试代码：注意，用户名和密码常用 Properties 字典表示，作为附加信息传递到 Driver。注意 JDBC 地址包含jdbc协议和mysql子协议，在子协议后有冒号和双斜杠，然后是数据库地址、数据库名。如果是本地3306，可以省略 localhost:3306，直接三个斜杠，跟上数据库名称即可。

```java
Driver driver = new com.mysql.jdbc.Driver();
String url = "jdbc:mysql://localhost:3306/log";
//注意，如果是本地地址和本地端口，可以使用:jdbc:mysql:///log 简写
Properties info = new Properties();
info.put("user","corkine");
info.put("password","password");

Connection connection = driver.connect(url,info);
System.out.print(connection);
```

**使用外部属性文件通过反射获取连接**

一个稍微进化的版本是：从属性文件中获取用户名、密码等数据库配置信息，这里的 InputStream 使用了 FileInputStream 而不是 `InputStream in = new JDBCTools().getClass().getClassLoader().getResourceAsStream("jdbc.properties");` 的原因是，我们希望最后的程序打包成 jar，同时属性文件放在外部。注意这里的 Driver 是通过反射 `Class.forName("xxx").newInstance()` 来进行创建的。这种创建方式会转型为 Dirver 接口类。

```java
public static Connection getBasicConnection() throws Exception {
    InputStream in = new FileInputStream(System.getProperty("user.dir") +File.separator + "jdbc.properties");
    Properties properties = new Properties();
    properties.load(in); in.close();

    Properties info = new Properties();
    info.put("user",properties.getProperty("user"));
    info.put("password",properties.getProperty("password"));

    Driver driver = (Driver) Class.forName(properties.getProperty("driver")).newInstance();
    Connection connection = driver.connect(properties.getProperty("url"),info);
    return connection;
}
```

**使用DriverManager获取连接**

相比较使用 Driver 获取连接，使用 DriverManager 的好处显而易见：可以同时管理多个数据库驱动程序驱动的数据库。其使用方法很简单 `DriverManager.getConnection(url,user,password)` 即可。需要注意的是，在获取连接前需要实例化驱动：

```
For example, the following code fragment returns the runtime Class descriptor for the class named java.lang.Thread:
Class t = Class.forName("java.lang.Thread") //A call to forName("X") causes the class named X to be initialized.
```

DM 只需要初始化驱动即可，而不需要获取此引用。其工作原理如下：当使用 getConnection 的时候，会通过 `getConnection(var0, var3, Reflection.getCallerClass());` 重载，之后会找到这个 Class，然后委托其获得连接：`Connection var7 = var6.driver.connect(var0, var1);`

```java
public static Connection getManagerConnection() throws Exception {
    InputStream in = new FileInputStream(System.getProperty("user.dir") + File.separator + "jdbc.properties");
    Properties properties = new Properties();
    properties.load(in);

    Properties info = new Properties();
    info.put("user",properties.getProperty("user"));
    info.put("password",properties.getProperty("password"));

    Class.forName(properties.getProperty("driver"));
    return DriverManager.getConnection(
                properties.getProperty("url"),
                properties.getProperty("user"),
                properties.getProperty("password"));
}
```

## 使用 Statement 对象进行增删改查

一个最基础的增、删、改操作如下：

```java
Connection connection = null;
Statement statement = null;
try {
    connection = getManagerConnection();
    statement = connection.createStatement();
    statement.executeUpdate(sql);
} catch (Exception e) {
    e.printStackTrace();
} finally {
    if (statement != null) {
        try {
            statement.close();
        } catch (SQLException e) {
            e.printStackTrace();
        }
    }
    if (connection != null) {
        try {
            connection.close();
        } catch (SQLException e) {
            e.printStackTrace();
        }
    }
}
```

很显然，造成这样代码的原因是 Java 的异常机制，因为数据库在失败时必须关闭，否则影响其余人连接。但是关闭也会出错，因此需要继续的 try catch。在这里接触了一个新的对象： Statement，其代表一次 SQL 操作。通过 `con.createStatement` 获取基本的 Statement 对象，通过 `executeUpdate` 进行更新/增加/删除操作。注意，statement 有两个子类 `CallableStatement, PreparedStatement` 前者用于函数调用，后者用于预编译语法调用。

为了简化操作，我们把处理异常的关闭 Statement、Connection 的操作放在一个单独的类方法中：

```java
public static void release(ResultSet resultSet,Statement statement,Connection connection) {
    if (resultSet != null) { ... }
    if (statement != null) { ... }
    if (connection != null) { ... }
}
```

然后写一个基础的 execute 方法：

```java
public static void execute(String sql){
    Connection connection = null;
    Statement statement = null;
    try {
        connection = getManagerConnection();
        statement = connection.createStatement();
        statement.executeUpdate(sql);
    } catch (Exception e) {
        e.printStackTrace();
    } finally {
        release(statement,connection);
    }
}
```

最后用这两个方法提供 delect、insert 方法：

```java
public static void delect(String sql) { execute(sql);}
public static void insert(String sql) { execute(sql);}
```

## 使用 ResultSet 对象管理返回值

至于查询的话，需要接触 ResultSet 对象。

```java
public static ResultSet query(String sql) {
    Connection connection = null;
    Statement statement = null;
    try {
        connection = getManagerConnection();
        statement = connection.createStatement();
        return statement.executeQuery(sql);
    } catch (Exception e) {
        e.printStackTrace();
    } finally {
        release(statement,connection);
    }
    return null;
}
```

注意，这里使用的是 `executeQuery` 而不是 `executeUpdate` 方法。返回的是 `ResultSet` 对象。

下面是这个对象的使用方法：

```java
public static void printQuery(String sql) {
    Connection connection = null;
    Statement statement = null;
    ResultSet set = null;
    try {
        connection = getManagerConnection();
        statement = connection.createStatement();
        set =  statement.executeQuery(sql);
        while (set.next()) {
            int id = set.getInt(1);
            String name = set.getString(2);
            String email = set.getString(3);
            Date birth = set.getDate(4);
            System.out.printf("id: %s\nname: %s\nemail: %s\nbirth: %s\n\n",id,name,email,birth);
        }
    } catch (Exception e) {
        e.printStackTrace();
    } finally {
        JDBCTools.release(set,statement,connection);
    }
}
```

注意到，这里的 ResultSet 方法，其可以通过 `next()` 进行指针的遍历（类似于迭代器）。每次迭代返回结果集的一行。对于每行，通过 getXXX 来获取对应结果，传入参数可以为列名，或者为列的编号，编号从1开始。至于类型，则是通过强转完成的。如果不匹配或者无法强转，则报错。

需要注意的是，ResultSet 也需要进行关闭，并且在 Statement、Connection 关闭后无法获取。因此这里 release 了三个对象，并且是按照顺序进行的。 Java 和 SQL 的类型对应如下：

```
 * boolean - bit
 * short - smallint
 * int - integer
 * long - bigint
 * string - char, varchar, long varchar
 * byte - array binary, var binary
 * sql.date - date
 * sql.time - time
 * sql.timestamp - timestamp
```

# JDBC 高级技术

## 使用 prepareStatement 安全查询

使用基本的 Statement 容易被 SQL 注入，安全性很差。而通过子类 prepareStatement 不是采用字符串拼接的方式，可以避免注入，并且当执行多条相同语句的时候，可以避免编译耗时，直接执行，速度更优。

pS 的 SQL 语句如下：`INSERT into customers (name, email, birth) values (?,?,?,?)` 其中问号表示占位符，表示需要填入的一个元素。

```java
public static void update(String sql, Object...args) {
    Connection connection = null;
    PreparedStatement statement = null;
    try {
        connection = getManagerConnection();
        statement = connection.prepareStatement(sql);
        for (int i = 0; i < args.length; i++) {
            //注意，setXXX 的索引是从1开始的
            statement.setObject(i+1,args[i]);
        }
        statement.executeUpdate();
    } catch (Exception e) {
        e.printStackTrace();
    } finally {
        JDBCTools.release(null,statement,connection);
    }
}
```

以上是改进过的 update 方法，statment 通过 `con.prepareStatement(sql)` 获取，需要传入准备好的一条 sql 语句，然后通过 setXXX(i,obj) 来设置指定类型的数据到索引 i 处。注意， i 从1开始计，表示第 i 个问号（从左到右）。一般而言，我们通过可变数组 args 和 for 循环来 setObject 到 statement。当设置好占位符后，通过 executeUpdate/Query 进行更新。

## 开启事务管理

事务指的是一次元操作，在以上的例子中，我们每次执行一条 SQL 语句都需要连接一次数据库，并且提交保存，然后再进行连接、保存。这样很消耗资源，并且不能撤销。如果在两个SQL语句间隔发生错误，那么我们将无法在本地撤销第一条SQL语句，这就造成了严重的问题。因此，我们可以通过事务：一次执行多条SQL语句，来避免中断的问题，当发生错误，我们进行回滚，当操作成功，则将多条同时提交。

事务区别于批处理，批处理指的是在本地写多条语句，然后一次交给服务器。而事务指的是每一条都上传服务器，但是在完全成功之前，可以撤销事务内的所有SQL语句。

事务的写法大致和之前类似，区别在于，<u>现在执行查询时，不关闭 con 对象，只关闭 rs 和 stat 对象。并且在每次查询的时候，都需要传入 con 引用。在执行查询前需要关闭 con 的自动提交，在完全结束后，需要对 con 进行总的提交。</u>很多时候，会不经意间忘记进行提交，或者关闭自动提交，就会出现问题。

通过 `con.setAutoCommit` 关闭自动提交，通过 `con.commit` 进行提交。如果发生错误，通过 `con.rollback` 进行回退。

```java
Connection connection = null;
try {
    connection = JDBCTools.getManagerConnection();
    connection.setAutoCommit(false); //关闭自动提交
    connection.setTransactionIsolation(Connection.TRANSACTION_READ_COMMITTED); //设置事务隔离级别

    String sql_a = "update customers set name = 'fuck' where id = 521";
    JDBCTools.update(connection,sql_a);
     
    String sql_b = "update customers set name = 'LiuJin' where id = 521";
    JDBCTools.update(connection,sql_b);

    connection.commit();
} catch (Exception e) {
    e.printStackTrace();
    try {
        connection.rollback(); //错误时进行回退
    } catch (SQLException e) {
        e.printStackTrace();
    }
} finally {
    JDBCTools.release(null,null,connection);
}
```

**事务的隔离级别**

- 读未提交（未提交的数据也可以读）
- 读已提交（只能读取已提交的数据）
- 可重复读（重复读取一个数据不变）
- 串行化（拒绝第二个用户用表）

Oracle默认数据库为读已提交的数据（不能避免多行、多次读数据不同，但可以避免读到未提交数据），MySQL 默认为可重复读（不能避免多行，但避免了多次读取值不同）

一般在 MySQL 中设置即可，不需要使用代码：

```mysql
show variables like '%isolation%';
help isolation
set [global|session] transaction isolation level xxxx;
```

**基于事务的更新方法**

```java
public static void update(Connection con, String sql, Object...args) {
    Connection connection = con;
    PreparedStatement statement = null;
    try {
        statement = connection.prepareStatement(sql);
        for (int i = 0; i < args.length; i++) {
            statement.setObject(i+1,args[i]);
        }
        statement.executeUpdate();
    } catch (Exception e) {
        e.printStackTrace();
    } finally {
        //注意这里不能关闭connection，因为是事务
        JDBCTools.release(null,statement,null);
    }
}
```

## 批处理 SQL 语句

批处理区别于事务，其所有语句同时提交给 SQL 服务器。但是又区别于多次执行单条语句，批处理在一个 statement 下执行了多条语句，速度更快。常见的批处理用法是：对于大量数据写入，使用循环，当满足一定条件进行一次提交并且清空批处理，然后再循环-提交。直到最后完成。

```java
String sql_temp = "insert into test (name, email, birth) values (?,?,?)";
Connection connection = null;
PreparedStatement statement = null;
try {
    connection = JDBCTools.getManagerConnection();
    connection.setAutoCommit(false);
    statement = connection.prepareStatement(sql_temp);
    //使用addBatch和executeBatch来批量执行语句，常用于批量添加数据，比如：
    for (int i = 0; i < 10000; i++) {
        statement.setString(1,"test" + i);
        statement.setString(2,"xxx"  + i);
        statement.setString(3,"1999-10-01");
        statement.addBatch(); //没有参数的addBatch适用于PreparedStatement进行setXXX
        //积攒到一定程度执行一次，同时需要清空这些batch!!!!!这样能大幅度提高效率（多线程更新+遍历）
        if ((i+1) % 300 == 0) {
            statement.executeBatch();
            statement.clearBatch();
        }
    }
    statement.executeBatch(); //这里的executeBatch代替了executeUpdate/Query
    statement.clearBatch(); //随手清空是个好习惯
    connection.commit();
} catch (Exception e) {
    e.printStackTrace();
    //如果出错，则进行回滚
    JDBCTools.rollback(connection);

} finally {
    JDBCTools.release(null,statement,connection);
}
```

在上述示例中，我们使用 `st.addBatch` 添加批处理语句，使用 `st.executeBatch` 执行批处理语句，使用  `st.clearBatch` 清空当前添加的批处理语句。注意，对于预处理的 statment，每次 addBatch 会自动清空对于占位符的赋值。因此我们可以在 for 循环中先填充占位符，然后添加批处理，接着清空占位符，循环往复。为了加快速度，往往在循环中满几百次时执行一次提交操作（executeBatch相当于query）。之后需要清空当前批处理池！！！！！这点很重要。

在最后防止有未完成的批处理，再执行一次 exe 和 clear。因为我们开启了事务，所以最后 con.commit() 进行提交即可。

总而言之，批处理的步骤是 `setObject - addBatch - ↻( executeBatch - clearBatch) - commit`

## 使用数据池

在之前的代码中，每次连接我们都使用了一个 con 对象，尽管事务共享了一个 con 对象，但是每次事务也需要进行文件读取、数据库连接状态查询和处理，耗时很大。现代几乎没有直接通过 Driver、DriverManager 进行数据库连接的EE程序，所有程序均通过数据池进行连接。

数据池指的是预先放置很多连接的对象，当需要时，直接请求一个已经存在的对象即可。这大大加快了访问数据库的能力。数据池会在连接不够时自动补充连接。常见的数据池有 DBCP 和 C3P0。其中后者速度更快，更强大。

**通过配置文件或者硬编码使用 DBCP**

对于 DBCP 而言，其依赖 pools 和 dbcp 包，是 apache 的 common 计划产物。通过 basicdatabase 获取 datasource，通过设置 username、password、url、driverclassname 来进行设置，通过 getConnection 获取连接。

其中 datasource 是 javax.sql 的一个标准数据池接口。

```java
BasicDataSource source = new BasicDataSource();

source.setUsername("corkine");
source.setPassword("password");
source.setUrl("jdbc:mysql:///log");
source.setDriverClassName("com.mysql.jdbc.Driver");

source.setInitialSize(10); //初始化的连接数
source.setMaxTotal(50); //设置最大的连接数
source.setMaxIdle(20); //在空闲时最多可以保留的连接
source.setMinIdle(5); //在空闲时最少可以保留的连接

source.setMaxWaitMillis(1000 * 5); //最长的等待时间，超过则抛异常（ms）

Connection connection = source.getConnection();
System.out.println(connection);
```

注意，我们常常需要设置驱动、地址、用户登录凭证，以及数据池专用的初始化连接数、最大最小、空闲最大最小连接数、最长等待时间（当现在到达最大连接数，但是没有连接可用的等待时间）。

通常而言，我们希望从外部加载配置文件。通过 `BasicDataSourceFactory.createDataSource(prop)` 获取 DataSource 对象。注意，这种方式有效的降低了和专有实现的耦合。在 properties 中的属性名为对应方法的 bean 名（去掉set，首字母小写即可）。

配置文件如下：

```
username=corkine
password=password
driverClassName=com.mysql.jdbc.Driver
url=jdbc:mysql:///log
maxWaitMillis=5000
maxTotal=5
```

下面的实验展示了当最大连接为5时，第六个连接等待的情况，3s后，程序为其分配了连接，因为小于最大超时5s，因此程序没有报错。

```java
InputStream in = new FileInputStream(System.getProperty("user.dir") + File.separator + "db.properties");
Properties properties = new Properties();
properties.load(in); in.close();
//通过这种方法（属性文件），可以有效的降低和common.dbcp包的耦合，只此一句→
DataSource dataSource = BasicDataSourceFactory.createDataSource(properties);

Connection c1 = dataSource.getConnection();
System.out.println(c1.getClass());
Connection c2 = dataSource.getConnection();
System.out.println(c2.getClass());
Connection c3 = dataSource.getConnection();
System.out.println(c3.getClass());
Connection c4 = dataSource.getConnection();
System.out.println(c4.getClass());
Connection c5 = dataSource.getConnection();
System.out.println(c5.getClass());
/*Connection c6 = dataSource.getConnection();
System.out.println(c6.getClass());*/ //出现异常，因为最大连接为5，但是超过5，并且超时

new Thread(() -> {
    Connection connection = null;
    try {
        connection = dataSource.getConnection();
        System.out.println(connection.getClass());
    } catch (SQLException e) {
        e.printStackTrace();
    }
}).start();

TimeUnit.SECONDS.sleep(3); //3s后方法运行完毕，此前的5个链接因为丢失索引所以被gc，因此新线程的连接在超时之前获得连接
```



**通过XML或者硬编码使用 C3P0**

C3P0 的用法和 DBCP 大致相同，同样需要设置 driverClass、jdbcUrl、user、password。C3P0 使用 `ComboPooledDataSource` 建立连接。

```java
ComboPooledDataSource dataSource = new ComboPooledDataSource();
dataSource.setDriverClass("com.mysql.jdbc.Driver");
dataSource.setJdbcUrl("jdbc:mysql:///log");
dataSource.setUser("corkine");
dataSource.setPassword("password");

dataSource.setMaxPoolSize(50);
dataSource.setMinPoolSize(5);

System.out.println(dataSource.getConnection());
```

C3P0 可以通过 XML 或者配置文件进行配置，xml 有默认的名称和查找位置，各种具名的配置组可供配置。通过 `ComboPooleddataSource` 而不是 Factory 进行连接，注意，这里默认查找类路径下的 c3p0-config.xml 文件，另可选配置文件。下面传入的 String 参数是XML中的配置组名称。

```java
DataSource dataSource = new ComboPooledDataSource("c3p0");
System.out.println(dataSource.getConnection());
```

```xml
<c3p0-config>
    <!--默认的配置-->
    <default-config>
        <property name="automaticTestTable">con_test</property>
        <property name="checkoutTimeout">30000</property>
        <property name="idleConnectionTestPeriod">30</property>
        <property name="initialPoolSize">10</property>
        <property name="maxIdleTime">30</property>
        <property name="maxPoolSize">100</property>
        <property name="minPoolSize">10</property>
        <property name="maxStatements">200</property>

        <user-overrides user="test-user">
            <property name="maxPoolSize">10</property>
            <property name="minPoolSize">1</property>
            <property name="maxStatements">0</property>
        </user-overrides>

    </default-config>

    <!--特定的配置-->
    <named-config name="c3p0">
        <property name="user">corkine</property>
        <property name="password">password</property>
        <property name="driverClass">com.mysql.jdbc.Driver</property>
        <property name="jdbcUrl">jdbc:mysql:///log</property>

        <!--如果数据库连接数不足，一次补足的连接个数-->
        <property name="acquireIncrement">50</property>
        <!--初始化数据库最大数量、最小连接数、最大连接数-->
        <property name="initialPoolSize">5</property>
        <property name="minPoolSize">5</property>
        <property name="maxPoolSize">100</property>

        <!--C3P0数据库连接池可以维护的 Statement 个数，每个 Connection 可用的 Statement-->
        <property name="maxStatements">20</property>
        <property name="maxStatementsPerConnection">5</property>

        <!--对于某个用户重写规则-->
        <user-overrides user="master-of-the-universe">
            <property name="acquireIncrement">1</property>
            <property name="initialPoolSize">1</property>
            <property name="minPoolSize">1</property>
            <property name="maxPoolSize">5</property>
            <property name="maxStatementsPerConnection">50</property>
        </user-overrides>
    </named-config>
</c3p0-config>
```

C3P0 的 XML 配置可选很多具名的配置策略，针对不同的 user 可使用不同的针对性策略。此外，可设置一次补足的连接个数、每个连接可用的stat个数、一共可用的stat个数，其粒度更细，灵活性更高。

数据池在我们的工具中如下使用，因为创建数据池耗费大量资源，所以只保留一个即可（如果影响程序启动，则使用单例模式）。

```java
private static DataSource dataSource;
static { dataSource = new ComboPooledDataSource("c3p0");}
public static Connection getConnection() throws Exception { return dataSource.getConnection();}
```

## 获取自增主键

在 MySQL 中，经常会因为插入而需要返回对其引用的情况，在这种情况下，对于 stat 获取时，可添加参数，要求进行插入时返回主键的值，`Statement.RETURN_GENERATED_KEYS`。通过 `stat.getGeneratedKey` 获取 ResultSet，第一行的第一列即是 id，通过 `next(), getInt(1)` 来获取。注意，只有在进行插入（executeUpdate）时、选择了主键自增时才会返回这个值。

```java
String sql = "insert into customers (id,name,email,birth) values (55,'Lili','lili@muninn.cn','2018-01-02')";
Connection connection = null;
PreparedStatement statement = null;
try {
    connection = JDBCTools.getManagerConnection();
    statement = connection.prepareStatement(sql, Statement.RETURN_GENERATED_KEYS);
    statement.executeUpdate();
    ResultSet set = statement.getGeneratedKeys(); //仅有executeUpdate才能获得主键值。
    //常用于插入数据，并且需要获取此数据的引用的情况，对于返回的ResultSet读取getObject(1)第一列即可。
    if (set.next()) System.out.println(set.getObject(1));
} catch (Exception e) {
    e.printStackTrace();
} finally {
    JDBCTools.release(null,statement,connection);
}
```

## 大文件存储

Oracle 中大文件类型为 LOB，其分为内部 LOB 和外部 LOB，其中内部 LOB 分为 BLOB, CLOB, NCLOB，分别指的是二进制、单字节字符、多字节字符。外部 LOB 仅包括 BFILE 类型，指的是文件保存在文件系统，而引用保存在数据库，不参与事务，只读。

MySQL 有 BLOB, 其分为 TinyBlob（<255字节），Blob（<65k），MediumBlob（<16M）,LongBlob(<4G)。如果存储文件过大，则性能下降。

blob 通过 st.setBlob 进行写入，通过 st.readBlob 进行读出。

```java
@Test public void learnSetBlob() {
    String sql = "insert into customers (id,name,email,birth,picture) values " +
            "(56,'Liyang','ly@muninn.cn','2018-11-02',?)";
    Connection connection = null;
    PreparedStatement statement = null;
    try {
        connection = JDBCTools.getManagerConnection();
        statement = connection.prepareStatement(sql);
        InputStream ins = new FileInputStream(System.getProperty("user.dir") + File.separator + "blob1.png");
        statement.setBlob(1,ins);
        //对于setXXX，且使用了prepareStatement，必须提供插入的？顺序（第几个问号）
        statement.executeUpdate();
    } catch (Exception e) {
        e.printStackTrace();
    } finally {
        JDBCTools.release(null,statement,connection);
    }
}
```

对于读出，通过 blob.getBinaryStream 获取二进制流。可用 BufferOutputStream，也可以直接按大小读数据（通过 `byte[] 和 len`）。

```java
@Test public void readBlob() {
    String sql = "select * from customers where id = 56 ";
    Connection connection = null;
    PreparedStatement statement = null;
    ResultSet set = null;
    try {
        connection = JDBCTools.getManagerConnection();
        statement = connection.prepareStatement(sql);
        statement.executeQuery();
        set = statement.getResultSet();
        if (set.next()) {
            String name = set.getString(2);
            String email = set.getString(3);
            Date date = set.getDate(4);
            Blob blob = set.getBlob(5);
            System.out.printf("name: %s, email %s, date %s, blob %s", name,email,date,blob);

            //输出Blob，使用getBinaryStream方法获得InputStream，使用BufferedOS或者手动使用byte[] buffer 进行读写到文件。
            InputStream in = blob.getBinaryStream();
            OutputStream out = new FileOutputStream("flower.jpg");
            byte[] buffer = new byte[1024]; int len;
            while ((len = in.read(buffer)) != -1) out.write(buffer,0,len);
            in.close(); out.close();
        }
    } catch (Exception e) {
        e.printStackTrace();
    } finally {
        JDBCTools.release(set,statement,connection);
    }
}
```

## 通过 Scanner 获取输入

Scanner 相比 InputStream 更方便。传入 Sys.in 即可获取输入。当调用 nextXXX 的时候，会自动将获取的东西强转成指定类型。如果没有东西，则阻塞输入，直到得到输入为止。如果得到了一个指定类型的东西，则删除间隔符（默认空白符），指向下一个元素。如果得到的类型不是（不能强转到）指定类型，则报错。

因此，常常通过 `hasNextXXX()`，进行是否存在下一个判断，默认的 `hasNext()` 方法接受任何除了空白之外的输入，同时，这些判断不会阻塞输入，如果没有输入，则直接返回。

可以使用 `useXXX` 来指定分隔符取代默认的空白分隔符。

```java
Scanner scanner = new Scanner(System.in);
//nextXXX方法会阻塞输入，一直到获取到为止。如果指针的next不是int，则报不匹配错误
//其实就是一个next方法，然后进行类型强制转换而已，然后去除分隔符，将指针指向分隔符后
System.out.print(scanner.nextInt());
//hasNextInt 指的是指针下一个元素是否可转化成int的意思，如果不能则直接为空，即便几个元素以后有int类型数据
while (scanner.hasNextInt()) {
    System.out.printf("get int: %s\n",scanner.nextInt());
}
```

# 基于 JDBC 的 DAO 方法

Data Access Object 指的是用于 JDBC 数据库对象和 Java 对象的互相转换的类，一般 Java 程序通过这个 DAO 类来存储 Java 类型到数据库，而不直接和 JDBC 打交道。

Database 数据相比 Java 对象存在着扁平化的问题，数据粒度的问题、双向索引查找的问题，因此 DAO 作为数据库和程序交互皆苦，因为项目需求不同而不同，因此常常需要手动编写。最为复杂的 DAO 演变成了 ORM，催生了 Hibernate 这种复杂的框架。

为了快速的从数据库得到Java对象，在 JDBCUtil 中我们尝试建立这样的一种连接。

```java
public class Customer {
    private String name;
    private int id;
    private String email;
    private Date birth; 
    ... }
```

这个类含有四个属性（叫做属性而不是字段的意思是，它们都默认提供了EE规定的 set/get 公开方法），同时具有默认无参构造器、具有四个参数的构造器。

## DatabaseMetaData

```java
DatabaseMetaData metaData = connection.getMetaData();
System.out.printf("Database Info: \nURL: %s\nUserName: %s\nReadOnly: %s\n" +
                "ProductName: %s\nProductVersion: %s\nDriverName: %s\nDriverVersion: %s\n ",
        metaData.getURL(),metaData.getUserName(),metaData.isReadOnly(),metaData.getDatabaseProductName(),
        metaData.getDatabaseProductVersion(),metaData.getDriverName(),metaData.getDriverVersion());
```

DBMD 从 con.getMetaData() 获得。其包含了当前数据库地址、用户名、数据库版本、名称、驱动、驱动版本等连接源信息。

## ResultSetMetaData

```java
ResultSetMetaData setMetaData = set.getMetaData();
System.out.printf("SetData Info:\nColumnName1: %s\nColumnCount: %s\n" +
        "ColumnTypeName1: %s\nColumnDisplaySize1: %s\nisNullable1: %s\n" +
        "isAutoIncrement1: %s\n",
        setMetaData.getColumnName(1),setMetaData.getColumnCount(),setMetaData.getColumnTypeName(1),
        setMetaData.getColumnDisplaySize(1),setMetaData.isNullable(1),
        setMetaData.isAutoIncrement(1));
```

通过 ResultSet 的方法获取 ResultSetMeta 对象，从而获取结果集的属性信息，包括列数、行数、列名、类别、列的属性等。DAO 为了创建对象，必须知道这些信息：列别名、列类型。

## 使用 反射 构建 Java 对象

第一版的 DAO 如下所示，需要传入类、查询语句以及占位符参数信息，通过此方法可以获得对象。

注意，我们从 setMetaData 中获取了列数(getColumnCount)，列的别名（getColumnLabel）,列的类型（getObject(id).getClass）。这就具备了反射创建 POJO 的条件，我们通过 tClass.getMethod 获取对应参数类型的方法，然后通过 newInstance 创建无参对象，通过 method.invoke 来调用对象的方法，完成了属性的注入。

这种方法很笨，原因是，我们需要手动的查找 bean 的方法：set + bean 首字母大写。这种方法同时不能处理基本类型和包装器类型的转换。改进方法采用了 beanUtil 提供的反射。

```java
public static <T> T getObject(Class<T> tClass, String sql, Object...args) {
    T entity = null;
    Connection connection = null;
    PreparedStatement statement = null;
    ResultSet set = null;

    try {
        connection = JDBCTools.getManagerConnection();
        statement = connection.prepareStatement(sql);
        for (int i = 0; i < args.length; i++) {
            statement.setObject(i+1,args[i]);
        }
        set = statement.executeQuery();
        if (set.next()) {
            entity = tClass.newInstance();
            ResultSetMetaData setMetaData = set.getMetaData();
            for (int i = 0; i < setMetaData.getColumnCount(); i++) {
                //一个简单的反射，通过找到对应参数的method，调用invoke方法，传递参数到对象中去
                //这种方法要求Java数据类型和SQL的数据类型一一对应。
                String label = setMetaData.getColumnLabel(i+1);
                String beanLabel = "set"+label.toUpperCase().substring(0,1) + label.substring(1);
                Object value = set.getObject(label);
                Method method = tClass.getMethod(beanLabel,new Class[]{value.getClass()});
                method.invoke(entity,value);
            }
        }
    } catch (Exception e) {
        e.printStackTrace();
    } finally {
        release(set,statement,connection);
    }
    return entity;
}
```

注意到，这里当 set.next() 存在，也就是有至少一行数据的时候才构建对象，进行反射。这种方法只能获取第一行的对象。为了获取多行对象，提供容器，并且改 if 为 for 即可。

## 使用 BeanUtils 构建 Java 对象

BeanUtils 通过 setProperty 和 getProperty 设置和获取 bean 的属性值。注意：

BeanUtil并非针对字段填值，其会寻找 name 的 setName 方法，然后执行这个方法，它不在乎实际字段是 name2 还是 name3，只要有这个方法，就执行。

至于 get，则相当于寻找 getName 这个方法。BeanUtil 实际上是对于 public 的 set/get 方法进行操作，我们需要传入的是这些个方法对应JavaEE规范的那个属性名称，属性名称可以不存在，而由某些字段在内部代理。

相比较自己处理字符串，BeanUtil可以自动对于 set 的类型进行处理，这点要省去寻找 method,并且 invoke 的很多类型匹配精力。在DAO使用 BeanUtil，就不用Java对象和SQL类型完全的属性类型对应了。

换句话说，BeanUtils 也是通过反射手动构造和查询 method 来进行属性注入的。这意味着我们不一定要保存和属性名一样的字段名。

```java
Customer customer = new Customer();
BeanUtils.setProperty(customer,"name","Marvin");
System.out.println(customer);
System.out.println(BeanUtils.getProperty(customer,"name"));
```

# 基于 JDBC 的 DAO 模板

现实中，经常为项目创建 DAO 模板，以用来进行增删改查，其相当于我们在上面介绍的 JDBCUtils。现在，我们将增删改查从 JDBCUtils 中分离，使得其只管理数据库属性读取、实例化保存数据库池 DataSource、提供基本的异常处理包装 rollback、release。换句话说，提供可用的数据库连接 Connection 对象的类。

一个 DAO 的经常性工作有四：

- 更新、删除语句的执行
- 查询并返回一个对象
- 查询并返回一组对象
- 查询并返回统计值

其接口如下：

```java
public interface BasicDAO<T> {
    void update(Connection con, String sql, Object... args);
    List<T> getForList(Connection con, String sql, Object... args);
    T get(Connection con, String sql, Object... args) throws SQLException;                              
    <E> E getForValue(Connection con, String sql, Object... args);
    void batch(Connection con, String sql, Object... args);
}
```



## 更新和删除

```java
public void update(String sql, Object...args) {
    Connection connection = null;
    PreparedStatement statement = null;
    try {
        connection = JDBCTools.getConnection();
        statement = connection.prepareStatement(sql);
        for (int i = 0; i < args.length; i++) statement.setObject(i+1,args[i]);
        statement.executeUpdate();
    } catch (Exception e) {
        e.printStackTrace();
    } finally {
        JDBCTools.release(null,statement,connection);
    }
}
```

## 基于 BeanUtils 的查询

```java
public <T> T get(Class<T> tClass, String sql, Object... args) {
    T entity = null;
    Connection connection = null;
    PreparedStatement statement = null;
    ResultSet set = null;

    try {
        connection = JDBCTools.getConnection();
        DatabaseMetaData metaData = connection.getMetaData();
        statement = connection.prepareStatement(sql);
        for (int i = 0; i < args.length; i++) {
            statement.setObject(i+1,args[i]);
        }
        set = statement.executeQuery();
        if (set.next()) {
            entity = tClass.newInstance();
            ResultSetMetaData setMetaData = set.getMetaData();
            for (int i = 0; i < setMetaData.getColumnCount(); i++) {
                String label = setMetaData.getColumnLabel(i+1);
                Object value = set.getObject(label);
                BeanUtils.setProperty(entity,label,value);
            }
        }
    } catch (Exception e) {
        e.printStackTrace();
    } finally {
        JDBCTools.release(set,statement,connection);
    }
    return entity;
}
```

## 查询并返回对象列表

```java
public <T> List<T> getList(Class<T> tClass, String sql, Object... args) {
    List<T> list = new ArrayList<>();
    T entity = null;
    Connection connection = null;
    PreparedStatement statement = null;
    ResultSet set = null;

    try {
        connection = JDBCTools.getConnection();
        DatabaseMetaData metaData = connection.getMetaData();
        statement = connection.prepareStatement(sql);
        for (int i = 0; i < args.length; i++) {
            statement.setObject(i+1,args[i]);
        }
        set = statement.executeQuery();
        while (set.next()) {
            entity = tClass.newInstance();
            ResultSetMetaData setMetaData = set.getMetaData();
            for (int i = 0; i < setMetaData.getColumnCount(); i++) {
                String label = setMetaData.getColumnLabel(i+1);
                Object value = set.getObject(label);
                BeanUtils.setProperty(entity,label,value);
            }
            list.add(entity);
        }
    } catch (Exception e) {
        e.printStackTrace();
    } finally {
        JDBCTools.release(set,statement,connection);
    }
    return list;
}
```

## 获取单个统计值

```java
public <E> E getForValue(String sql, Object... args) {
    E entity = null;
    Connection connection = null;
    PreparedStatement statement = null;
    ResultSet set = null;

    try {
        connection = JDBCTools.getConnection();
        DatabaseMetaData metaData = connection.getMetaData();
        statement = connection.prepareStatement(sql);
        for (int i = 0; i < args.length; i++) {
            statement.setObject(i+1,args[i]);
        }
        set = statement.executeQuery();
        if (set.next()) return (E) set.getObject(1);
    } catch (Exception e) {
        e.printStackTrace();
    } finally {
        JDBCTools.release(set,statement,connection);
    }
    return entity;
}
```

# 基于 DBUtils 的 DAO 模板

## DBUtils 和 ResultSetHandler 介绍

DBUtils 是一个用来简化 JDBC 开发的工具包，属于 apache 的 common 系列。其大致和我们的 JDBCUtils 类似，不过是线程安全的，性能也更好。实际开发经常基于 DBUtils 来创建 DAO。当然，为了管理数据池、用户凭证、提供错误处理，我们在写 DAO 时同时使用了 JDBCUtils 自编类。

DBUtils 基于 QueryRunner 对象，一般将其实例化为一个对象作为类字段：

```java
QueryRunner runner = new QueryRunner();
```

对于更新，只需要执行 runner.update 即可，传入连接、语句、占位符参数即可。

```java
@Test public void updateLearn() {
    String sql = "delete from customers where id in (?,?)";
    Connection connection = null;
    try {
        connection = JDBCTools.getConnection();
        runner.update(connection,sql,10,55);
    } catch (Exception e) {
        e.printStackTrace();
    }
}
```

对于插入，执行 runner.insert 即可。需要注意，必须提供 ResultSetHandleer 处理返回的数据（？主键）这里留空。

```java
@Test public void insertLearn() {
    String sql = "insert into customers (name,email,birth) values ('CorM','cc@cc.cc','2001-01-22')";
    Connection connection = null;
    try {
        connection = JDBCTools.getConnection();
        runner.insert(connection, sql, new BaseResultSetHandler<Object>() {
            @Override
            protected Object handle() {
                return null;
            }
        });
    } catch (Exception e) {
        e.printStackTrace();
    } finally {
        JDBCTools.release(null,connection);
    }
}
```

对于查询，注意这里我们写了个匿名的 ResultSetHandler 方法，此方法用于截获返回对象ResultSet，并且对 Set进行处理，然后进行返回（相当于代理）。在这里很方便我们自己去使用 BeanUtils 构造对象，这里直接写死了获取的数据，并且直接通过构造器构造，这是为了展示。

```java
@Test public void queryLearn() {
    Connection connection = null;
    try {
        String sql = "select * from customers";
        connection = JDBCTools.getConnection();
        List<Customer> customers = runner.query(connection, sql, resultSet -> {
            //handle方法的返回值作为了 query 方法的返回值
            //这里一般用于截获ResultSet并且构造Java对象，返回此对象或者对象集合
            List<Customer> customerList = new ArrayList<>();
            while (resultSet.next()) {
                Integer id = resultSet.getInt(1);
                String name = resultSet.getString(2);
                String email = resultSet.getString(3);
                Date date = resultSet.getDate(4);
                Customer customer = new Customer(name,id,email,date);
                customerList.add(customer);
            }
            return customerList;
        });
        System.out.println(customers);
    } catch (Exception e) {
        e.printStackTrace();
    } finally {
        JDBCTools.release(null,connection);
    }
}
```

BeanUtils 提供了一些默认的 Handler，封装了 BeanUtils，可直接用来获取 bean，最基本的 beanHander 需要传入 .class 对象，然后会自动填充 bean，返回对象，非常方便！！！！

```java
/**beanHandler 把结果集的第一条创建对应类型的对象，并且返回此对象*/
@Test public void beanHandlerLearn() {
    Connection connection = null;
    try {
        connection = JDBCTools.getConnection();
        String sql = "select id, name, email, birth from customers where id = ?";
        Customer customer = runner.query(connection,sql,new BeanHandler<>(Customer.class),521);
        System.out.println(customer);
    } catch (Exception e) {
        e.printStackTrace();
    } finally {
        JDBCTools.release(null,connection);
    }
}
```

BeanListHandler 则提供了一个 bean 的列表

```java
/**beanListHandler 把结果集的所有结果创建对应类型的对象，并且返回此对象集合
 * 此集合绝对不为null，但可能只包含一个对象，或者为空列表*/
@Test public void beanListHandlerLearn() {
    Connection connection = null;
    try {
        connection = JDBCTools.getConnection();
        String sql = "select id, name, email, birth from customers";
        List<Customer> customers = runner.query(connection,sql,new BeanListHandler<>(Customer.class));
        System.out.println(customers);
    } catch (Exception e) {
        e.printStackTrace();
    } finally {
        JDBCTools.release(null,connection);
    }
}
```

BeanMapHandler 则提供了一个 bean 的“名称”而不是“Label”作为 key，其值作为 value 的 Map

```java
/**beanMapHandler 只返回第一条记录对应的Map对象，key是 SQL 查询的列名Name（不是别名Label）
 * value是对应SQL对象*/
@Test public void beanMapHandlerLearn() {
    Connection connection = null;
    try {
        connection = JDBCTools.getConnection();
        String sql = "select id, name, email, birth from customers";
        Map<String,Object> map = runner.query(connection,sql,new MapHandler());
        System.out.println(map);
    } catch (Exception e) {
        e.printStackTrace();
    } finally {
        JDBCTools.release(null,connection);
    }
}
```

BeanMapListHandler 则提供了 Map 的 List，对应多行。

```java
/**beanMapListHandler 返回所有记录对应的Map对象，key是 SQL 查询的列名Name（不是别名Label）
 * value是对应SQL对象*/
@Test public void beanMapListHandlerLearn() {
    Connection connection = null;
    try {
        connection = JDBCTools.getConnection();
        String sql = "select id, name, email, birth from customers";
        List<Map<String,Object>> map = runner.query(connection,sql,new MapListHandler());
        System.out.println(map);
    } catch (Exception e) {
        e.printStackTrace();
    } finally {
        JDBCTools.release(null,connection);
    }
}
```

scalarHandler 默认返回第一列的值，对应统计值查询：

```java
/**scalarHandler 默认返回第一列的值*/
@Test public void scalarHandlerLearn() {
    Connection connection = null;
    try {
        connection = JDBCTools.getConnection();
        String sql = "select max(id) from customers";
        Object obj = runner.query(connection,sql,new ScalarHandler<>());
        System.out.println(obj);
    } catch (Exception e) {
        e.printStackTrace();
    } finally {
        JDBCTools.release(null,connection);
    }
}
```

## 配合 JDBCUtils 和 DBUtils 的 DAO

配合包含了数据源和用户凭证的 JDBCUtils，基于 DBUtils 的 DAO 很好写，以下提供了一个通用模板。可能唯一的难点在于，泛型类型的获取，因为在 BeanHandler 需要传入此类型。通过以下方法：`(Class<T>) ((ParameterizedType)getClass().getGenericSuperclass()).getActualTypeArguments()[0];` 来获取泛型类型。

```java
public class JdbcDaoImpl<T> implements BasicDAO<T> {
    private QueryRunner runner = null;
    private Class<T> type;
    public JdbcDaoImpl() {
        runner = new QueryRunner();
        type = (Class<T>) ((ParameterizedType)getClass().getGenericSuperclass()).getActualTypeArguments()[0];
    }
    @Override
    public T get(Connection con, String sql, Object... args) throws SQLException {
        return runner.query(con,sql,new BeanHandler<>(type),args);
    }
... }
```

更新历史：

2018-10-04 完成基本的 JDBC 学习

2018-10-05 完成大文件、预处理、事务、批处理、DAO 接口的三种实现部分

2018-10-06 撰写本文